In [25]:
import pandas as pd
import numpy as np
import os
FILE = os.path.join(os.getcwd(), '11_14_check.xls')
df0 = pd.read_excel(FILE, header=1)

In [2]:
df0

,料品,品名,库位,批号,出,入
0,100101-05,太粮王泰国茉莉香米5,4A00,191119006-18,4.0,4.0
1,100101-05,太粮王泰国茉莉香米5,4AH0,191119006-18,3.0,3.0
2,100101-25,太粮王泰国茉莉香米25,4AH5,200917050,10.0,NaN
3,100101-25,太粮王泰国茉莉香米25,4AH4,200917050,NaN,10.0
4,100101-25,太粮王泰国茉莉香米25,4AH5,201024003,40.0,40.0
...,...,...,...,...,...,...
701,180115-74,黑米0.4*30,4A00,200908105,39.0,NaN
702,180115-74,黑米0.4*30,4AH0,200908105,1.0,NaN
703,180115-74,黑米0.4*30,4A00,200908101,NaN,158.0
704,180115-74,黑米0.4*30,4AH1,200908105,NaN,24.0


In [19]:
df = df0.fillna(0).copy()
_add = list()
for g, _df in df.groupby(['品名', '库位', '批号']):
    _df_add = pd.DataFrame()
    o_i = _df['出'].sum() - _df['入'].sum()
    if o_i > 0:
        _df_add = pd.DataFrame([_df.iloc[0]])
        _df_add[['出', '入']] = [o_i, 0]
    elif o_i < 0:
        _df_add = pd.DataFrame([_df.iloc[0]])
        _df_add[['出', '入']] = [0, -o_i]     
    _add.append(_df_add)
df = pd.concat(_add)
df.to_csv('i_o.csv', encoding='utf-8-sig', index=0)

In [60]:
def check(o, i):
    # print(f'{o}\n{i}\n')

    checks = list()
    if o.sum() == i.sum():
        return checks
    elif o.sum() > i.sum():
        difference = o.sum() - i.sum()
        direction = 'o'
        more, less = o, i
    else:
        difference = i.sum() - o.sum()
        direction = 'i'
        more, less = i, o
        
    if less.sum() == 0:
        for position in more.index:
            amount = more[position]
            checks.append((position, amount, direction))
        return checks
    # print(f'需要盘点数量：{difference}， 盘点方向：{direction}')
    
    while difference >= more.iloc[0]:
        position = more.index[0]
        amount = more[position]
        checks.append((position, amount, direction))
        difference -= more.pop(position)
        
    # print(f'从大到小贪心盘出：{checks}')


    if difference > 0:
        position = more[more >= difference].index[-1]
        # print(f'如果盘点数量仍有剩余，找到最后一个数量充足的库位：{(position, difference, direction)}')
        checks.append((position, difference, direction))
        if more[position] == difference:
            more.pop(position)
        else:
            more[position] -= difference            
            more.sort_values(ascending=False)    
    return checks

In [62]:
checks = list()
for sku_batch, df_oi in df.groupby(['品名', '批号']):
    o, i = df_oi['出'], df_oi['入']
    o, i = o[o > 0], i[i > 0]
    c = check(o, i)
    if c:
        checks += c
display(len(checks), checks[0: 10])


279

[(566, 3199.0, 'i'),
 (567, 499.0, 'i'),
 (565, 3.0, 'i'),
 (563, 951.0, 'o'),
 (564, 2750.0, 'o'),
 (265, 30.0, 'o'),
 (267, 30.0, 'i'),
 (25, 4.0, 'o'),
 (27, 8.0, 'i'),
 (28, 4.0, 'o')]

In [64]:
df_move = df.copy()
display(df_move)
_check = list()
for idx, amount, direction in checks:
    if direction == 'o':
        _df_check = pd.DataFrame([df_move.loc[idx]])
        _df_check[['出', '入']] = [amount, 0]
        if df_move.loc[idx, '出'] == amount:
            df_move = df_move.drop(labels=idx)
        else:
            df_move.loc[idx, '出'] -= amount 
    elif direction == 'i':
        _df_check = pd.DataFrame([df_move.loc[idx]])
        _df_check[['出', '入']] = [0, amount]
        if df_move.loc[idx, '入'] == amount:
            df_move = df_move.drop(labels=idx)
        else:
            df_move.loc[idx, '入'] -= amount 
    _check.append(_df_check)
df_check = pd.concat(_check)
display(df_check, df_move)
df_check.to_csv('i_o_check.csv', encoding='utf-8-sig', index=0)
df_move.to_csv('i_o_move.csv', encoding='utf-8-sig', index=0)


,料品,品名,库位,批号,出,入
566,107008-10,(单面）宫粥珍珠米10,3A01,200928010,0.0,3199.0
567,107008-10,(单面）宫粥珍珠米10,3AP1,200928010,0.0,499.0
563,107008-10,(单面）宫粥珍珠米10,3AP1,201001011,951.0,0.0
564,107008-10,(单面）宫粥珍珠米10,3AP1,201001012,2750.0,0.0
565,107008-10,(单面）宫粥珍珠米10,4AH1,200928010,0.0,3.0
...,...,...,...,...,...,...
162,101303-10,（靓虾）软米10,4B02,201105010,4256.0,0.0
380,103906-10,（鲜仔）五星丝苗米10,4AH2,200820001,0.0,4.0
382,103906-10,（鲜仔）五星丝苗米10,4AH2,201105011,4.0,0.0
383,103906-35,（鲜仔）五星丝苗米5*5,3A01,201111013,33.0,0.0


,料品,品名,库位,批号,出,入
566,107008-10,(单面）宫粥珍珠米10,3A01,200928010,0.0,3199.0
567,107008-10,(单面）宫粥珍珠米10,3AP1,200928010,0.0,499.0
565,107008-10,(单面）宫粥珍珠米10,4AH1,200928010,0.0,3.0
563,107008-10,(单面）宫粥珍珠米10,3AP1,201001011,951.0,0.0
564,107008-10,(单面）宫粥珍珠米10,3AP1,201001012,2750.0,0.0
...,...,...,...,...,...,...
167,101303-10,（靓虾）软米10,4A03,201111007,0.0,64.0
170,101303-10,（靓虾）软米10,4AP5,201111007,0.0,820.0
168,101303-10,（靓虾）软米10,4A04,201111034,0.0,680.0
380,103906-10,（鲜仔）五星丝苗米10,4AH2,200820001,0.0,4.0


,料品,品名,库位,批号,出,入
249,101312-10,(禾美)靓虾王香软米10,4A02,201110003,4.0,0.0
250,101312-10,(禾美)靓虾王香软米10,4AH2,201110003,0.0,4.0
251,101312-85,(禾美)靓虾王香软米2.5*10,3A07,201108001,70.0,0.0
252,101312-85,(禾美)靓虾王香软米2.5*10,4AP1,201108001,0.0,70.0
266,101401-10,(紫色)米皇坊小农粘米10,4AH2,201031032,0.0,24.0
...,...,...,...,...,...,...
164,101303-10,（靓虾）软米10,4A02,201105047,0.0,7.0
170,101303-10,（靓虾）软米10,4AP5,201111007,0.0,210.0
166,101303-10,（靓虾）软米10,4AP6,201111007,210.0,0.0
383,103906-35,（鲜仔）五星丝苗米5*5,3A01,201111013,33.0,0.0


In [2]:
death = '101306-10,101420-25,103903-25,101301-85,105809-05,101601-25,150101-35,101402-10,101406-15,103906-35,150103-15,101301-08,160204-35,160101-10,107001-35,101406-35,150103-35,170103-10,150107-35,107001-32,100501-35,101502-15,100606-35,101301-35,107011-85,180113-74,101602-25,101308-25,107014-35,103909-15,103907-10,103911-15,100603-32,101312-85,101402-85'

In [3]:
death.split(',')

['101306-10',
 '101420-25',
 '103903-25',
 '101301-85',
 '105809-05',
 '101601-25',
 '150101-35',
 '101402-10',
 '101406-15',
 '103906-35',
 '150103-15',
 '101301-08',
 '160204-35',
 '160101-10',
 '107001-35',
 '101406-35',
 '150103-35',
 '170103-10',
 '150107-35',
 '107001-32',
 '100501-35',
 '101502-15',
 '100606-35',
 '101301-35',
 '107011-85',
 '180113-74',
 '101602-25',
 '101308-25',
 '107014-35',
 '103909-15',
 '103907-10',
 '103911-15',
 '100603-32',
 '101312-85',
 '101402-85']